<a href="https://colab.research.google.com/github/AugustvonMackensen/AI_colab/blob/main/tensor_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

텐서(tensor) : 동일한 자료형의 데이터를 저장하는 다차원 배열을 의미함
벡터는 1차원 텐서, 행렬은 2차원 텐서로 표현함.
텐서는 다차원 배열과 스칼라 값을 표현할 수 있다.

텐서는 자료형을 저장할 수 있는 가장 빠른 장치에 저장할 수 있게 함으로써 성능을 높일 수 있음

In [1]:
import tensorflow as tf
import numpy as np

상수 텐서

In [2]:
a = tf.constant(10.)
b = tf.constant([1, 2, 3, 4])
c = tf.constant([[[1, 2], [3, 4], [5, 6]],
                 [[7, 8], [9, 10], [11, 12]],
                [[13, 14], [15, 16], [17, 18]],
                [[19, 20], [21, 22], [23, 24]]], dtype=tf.float32)
print('a : dtype = ', a.dtype, '\n', a)
print('b : shape = ', b.shape, '\n', b)
print('c : device = ', c.device)

a : dtype =  <dtype: 'float32'> 
 tf.Tensor(10.0, shape=(), dtype=float32)
b : shape =  (4,) 
 tf.Tensor([1 2 3 4], shape=(4,), dtype=int32)
c : device =  /job:localhost/replica:0/task:0/device:GPU:0


변수 텐서

In [3]:
x = tf.Variable(10.)
y = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
z = np.array([[1., 3.], [2., 4.], [3., 5.]], dtype=np.float32)
print('x : dtype = ', x.dtype, '\n', x)
print('y : shape =', y.shape, '\n', y)
print('y : device = ', y.device)

x : dtype =  <dtype: 'float32'> 
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=10.0>
y : shape = (2, 3) 
 <tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
y : device =  /job:localhost/replica:0/task:0/device:GPU:0


변수 텐서에 값 대입하기

In [4]:
x.assign_add(20.) # assign_add : += 연산자의 역할을 함.
print('x = ', x.numpy())

x =  30.0


텐서에 대한 산술 연산 및 수학 함수

In [5]:
print('a * b = ', (a * tf.cast(b, tf.float32)).numpy())
print('tf.math.exp(y) = ', tf.exp(y))
print('tf.math.reduce_sum(c, axis=2) = ', tf.reduce_sum(c, axis=2))

a * b =  [10. 20. 30. 40.]
tf.math.exp(y) =  tf.Tensor(
[[  2.7182817   7.389056   20.085537 ]
 [ 54.59815   148.41316   403.4288   ]], shape=(2, 3), dtype=float32)
tf.math.reduce_sum(c, axis=2) =  tf.Tensor(
[[ 3.  7. 11.]
 [15. 19. 23.]
 [27. 31. 35.]
 [39. 43. 47.]], shape=(4, 3), dtype=float32)


tf.math.exp : e^x의 값을 구한다. tf.math들의 함수들은 tf.math 생략 가능

reduce_sum : c의 axis=2에 대해 reduce_sum을 구한다. c[i, j, :]에 대한 합계를 각각의 i,j에 대해 축소(reduction) 연산자 이용해 계산함.

행렬에 대한 선형대산 연산이 필요하면 tf.linalg 모듈에서 제공하는 다양한 함수 사용 가능

텐서에 대한 연산에 numpy 배열이 사용되었다면 이를 텐서로 묵시적인 형 변환을 하여 처리함.

역으로 numpy 배열을 위한 연산에 텐서가 사용되면 텐서를 묵시적으로 numpy 배열로 형 변환을 하여 처리함.

In [6]:
print('tf.linalg.matmul(y,z) = ')
print(tf.matmul(y, z))
print('np.matmul(y, z) = ')
print(np.matmul(y, z))

tf.linalg.matmul(y,z) = 
tf.Tensor(
[[14. 26.]
 [32. 62.]], shape=(2, 2), dtype=float32)
np.matmul(y, z) = 
[[14. 26.]
 [32. 62.]]


자동 미분

딥러닝 모델의 역전파 학습은 가중치와 바이어스 등 여러 가지 학습 요소에 대한 손실함수의 편미분을 계산해야 함

자동 미분을 이용하면, 모델을 설계한 후 가중치 등 훈련 대상에 대한 편미분을 하는 수식을 직접 유도하지 않아도 됨으로 매우 편리함

자동 미분을 하려면 정방향 진행(forward pass)동안 어떠한 연산이 이루어지는지를 기억했다가 역방향 진행(backward pass)를 하는 동안 연산을 역으로 거슬러 올라가는 과정이 필요함.

텐서플로는 tf.GradientTape API를 통해 이러한 과정을 처리하는데, tf.GradientTape 의 문맥을 생성 후 정방향 진행의 연산을 테이프에 기록하고, 이 테이프를 되감기하여 미분을 계산함. 이 때 테이프의 watch 메소드를 추적할 텐서를 지정

단, trainable 속성이 True(변수 텐서의 디폴트 값)인 변수 텐서는 기본적으로 추적 대상이므로 watch 메소드로 지정하지 않아도 됨
tf.GradientTape의 gradient 메소드를 사용하면 테이프에 기록된 연산 사용하여 편미분 계산 가능.

In [7]:
x1 = tf.Variable(3.)
x2 = tf.Variable(1., trainable=False)
with tf.GradientTape() as t:
  t.watch(x2)
  y = (x1 + 2 * x2) ** 2
dy_dx = t.gradient(y, [x1, x2]) # (x1 + 2 * x2)^2에 대한 편미분을 구함
print(f'dy/dx1 = {dy_dx[0]}') # x1에 대한 편미분
print(f'dy/dx2 = {dy_dx[1]}') # x2에 대한 편미분

dy/dx1 = 10.0
dy/dx2 = 20.0


자동 미분을 이용한 선형회귀 문제 학습

In [8]:
x = tf.constant([1., 3., 5., 7.]) # 독립변수(상수 텐서)
y = tf.constant([2., 3., 4., 5.]) # 종속변수(상수 텐ㅅ)
w = tf.Variable(1.) # 학습할 파라미터(변수 텐서)
b = tf.Variable(0.5) # 학습할 파라미터(변수 텐서)
learning_rate = 0.01 # 학습률
epochs = 1000 # 반복 횟수

학습 단계 처리 함수 정의

In [9]:
def train_step(x, y):
  with tf.GradientTape() as t:
    y_hat = w * x + b
    loss = (y_hat - y) ** 2
    grads = t.gradient(loss, [w, b])
    w.assign_sub(learning_rate * grads[0])
    b.assign_sub(learning_rate * grads[1])

w, b 학습시 손실함수를 w,b에 대해 각각 편미분을 구한 다음 학습률을 곱한 값을 기존의 w와 b에서 빼는 과정 반복

여기 코드에서는 손실함수는 오차 제곱으로 정의되었다.

학습표본 집합에 대한 반복 학습

In [10]:
for i in range(epochs):
  for k in range(len(y)):
    train_step(x[k], y[k])

학습된 파라미터 출력

In [11]:
print('w : {:8.5f}    b: {:8.5f}'.format(w.numpy(), b.numpy()))

w :  0.50000    b:  1.50000


학습된 파라미터를 이용한 모델 실행

In [12]:
f = 'x:{:8.5f}  -->   y:{:8.5f}'
for k in range(len(y)):
  y_hat = w * x[k] + b
  print(f.format(x[k].numpy(), y_hat.numpy()))

x: 1.00000  -->   y: 2.00000
x: 3.00000  -->   y: 3.00000
x: 5.00000  -->   y: 4.00000
x: 7.00000  -->   y: 5.00000


**그래프 실행 모드(graph execution)**

딥러닝 모델을 표현하게 되는 경우 즉시 실행 모드에서는 명령 단위로 텐서플로 연산을 실행하여 결과를 파이썬으로 가져오는 과정을 반복함으로써 많은 시간을 소비하게 됨

그래프 실행 모드는 데이터 흐름 형태로 표현되는 계산 구조를 나타내는 텐서플로 그래프(tf.graph)를 실행하는 방식으로 동작됨.

그래프는 사용 가능한 하드웨어에서 효율적으로 실행하도록 최적화하여 컴파일되므로 그래프를 실행하면 계산을 빠르게 할 수 있고, 병렬처리 활용할 수 있고, 다수의 장치에서 효율적으로 동작하게 만들 수 있다는 장점이 있음

특히 대규모 모델, 방대한 양의 데잍에 대해 반복 훈련을 하는 경우 실행 속도 및 메모리 사용 측면에서 더 우수한 성능을 발휘함.

자동적으로 그래프로 변환하여 그래프 실행 모드를 동작하게 하려면 tf.function(파이썬 코드를 파이썬에 독립적인 그래프로 변환하는 도구)를 호출하는데, 직접 호출 방식과 수식어로 사용되는 두 가지 방법이 있음.

1) 직접 호출 방식 : 일반적 파이썬 함수를 인수로 전달하여 tf.function 호출
tf.function은 그 결과를 파이썬이 호출할 수 있는 함수로 변환하여 돌려줌.
이 함수를 원래 함수를 사용하듯 사용하면 됨.

2)수식어 사용 방법 : @tf.function 으로 사용.

In [13]:
@tf.function
def train_step2(x, y):
  with tf.GradientTape() as t:
    y_hat = w * x + b
    loss = (y_hat - y) ** 2
    grads = t.gradient(loss, [w, b])
    w.assign_sub(learning_rate * grads[0])
    b.assign_sub(learning_rate * grads[1])

In [14]:
train_step_graph = tf.function(train_step2)
for i in range(epochs):
  for k in range(len(y)):
    train_step_graph(x[k], y[k])

In [15]:
print('w : {:8.5f}    b: {:8.5f}'.format(w.numpy(), b.numpy()))

w :  0.50000    b:  1.50000


In [16]:
f = 'x:{:8.5f}  -->   y:{:8.5f}'
for k in range(len(y)):
  y_hat = w * x[k] + b
  print(f.format(x[k].numpy(), y_hat.numpy()))

x: 1.00000  -->   y: 2.00000
x: 3.00000  -->   y: 3.00000
x: 5.00000  -->   y: 4.00000
x: 7.00000  -->   y: 5.00000


결과 분석 : 즉시 실행 모드에 비해 빠르게 실행되는 것을 확인할 수 있었음,